# HW Week 6 - Davis Club

---

Jeff Shamp, John Kellogg, Grace Han

---

From BB and the NetworkX documentation

In [15]:
import matplotlib.pyplot as plt
import networkx as nx
import networkx.algorithms.bipartite as bipartite
import warnings
from pyvis.network import Network

warnings.filterwarnings('ignore')
% matplotlib inline


G = nx.davis_southern_women_graph()
women = G.graph["top"]
clubs = G.graph["bottom"]

# Omitting the biadjacent matrix for brevity
# print("Biadjacency matrix")
# print(bipartite.biadjacency_matrix(G, women, clubs))

# project bipartite graph onto women nodes
W = bipartite.projected_graph(G, women)
print()
print("#Friends, Member")
for w in women:
    print(f"{W.degree(w)} {w}")

# project bipartite graph onto women nodes keeping number of co-occurence
# the degree computed is weighted and counts the total number of shared contacts
W = bipartite.weighted_projected_graph(G, women)
print()
print("#Friend meetings, Member")
for w in women:
    print(f"{W.degree(w, weight='weight')} {w}")


#Friends, Member
17 Evelyn Jefferson
15 Laura Mandeville
17 Theresa Anderson
15 Brenda Rogers
11 Charlotte McDowd
15 Frances Anderson
15 Eleanor Nye
16 Pearl Oglethorpe
17 Ruth DeSand
17 Verne Sanderson
16 Myra Liddel
16 Katherina Rogers
17 Sylvia Avondale
17 Nora Fayette
17 Helen Lloyd
16 Dorothy Murchison
12 Olivia Carleton
12 Flora Price

#Friend meetings, Member
50 Evelyn Jefferson
45 Laura Mandeville
57 Theresa Anderson
46 Brenda Rogers
24 Charlotte McDowd
32 Frances Anderson
36 Eleanor Nye
31 Pearl Oglethorpe
40 Ruth DeSand
38 Verne Sanderson
33 Myra Liddel
37 Katherina Rogers
46 Sylvia Avondale
43 Nora Fayette
34 Helen Lloyd
24 Dorothy Murchison
14 Olivia Carleton
14 Flora Price


We see from the degrees of members and member meetings that all of the woman have about the same number of friends, so this network isn't "held together" by a small number of "super-friends". Also with 18 total women and most women having around 15 friends, almost all the women know all the other women. We also see that there is a gradient of number of meetings each member attends. We do have several women who meet many many other women, this is likely due to the club they attend. We will visual this original 2-mode network and then the 1-mode projection. 

In [17]:
M = Network(height='800px',
            width='1000px',
            bgcolor='#222222', 
            font_color='white', 
            notebook=True)

M.barnes_hut()
for n in G.nodes():
    M.add_node(n)
for e in G.edges():
    M.add_edge(e[0], e[1])
M.show('network_2.html')

So it is clear that there are large and small clubs that serve has connectors between members. There are about four clubs that connect a large number of women and several smaller clubs that connect only a few new people to the larger network. 

In [14]:
C = Network(height='1000px',
            width='1000px',
            bgcolor='#222222', 
            font_color='white', 
            notebook=True)

C.barnes_hut()
for n in W.nodes():
    C.add_node(n)
for e in W.edges():
    C.add_edge(e[0], e[1])
C.show('network_2.html')

For the seven women in the middle of the network who have many (40 +) connections they attend at least one of the major clubs in addition to some smaller clubs. This makes gives them such a high weighted degree. 